In [26]:
import numpy as np

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

# 定义系数矩阵A和常数向量b
A = np.array([[3, 2, -1],
              [2, -2, 4],
              [-1, 0.5, -1]])
b = np.array([1, -2, 0])

# 使用numpy.linalg.solve求解
x = np.linalg.solve(A, b)

print("解为：", x)


adj_matrix = np.array([
    [-1, -1, 0, 0, 0, 0],
    [1, 0,   0, 0, 0,-1],
    [0, 0,   0,-1, 0, 1],
    [0 ,0 , -1 ,1 ,0, 0],
    [0 ,1 ,  1 ,0, 0, 0],
    [0 ,0 ,  0, 0 ,1, 0]
    
])

node_demands = np.array([0.1, 0.,0. ,0.1 , -0.,0])  # 示例需求，单位可以是立方米/秒等

# 构建增广矩阵
augmented_matrix = np.column_stack((adj_matrix, -node_demands))

# 计算系数矩阵A和增广矩阵的秩
rank_A = np.linalg.matrix_rank(adj_matrix)
rank_augmented = np.linalg.matrix_rank(augmented_matrix)

# 判断方程组是否有解
if rank_A == rank_augmented:
    print("方程组有解")
else:
    print("方程组无解")
    n = adj_matrix.shape[0]
    # 改变填充节点
    for i in range(n):
        for j in range(n):
            if adj_matrix[i, j] ==1: #!= 0:  # 只要存在边
                # idx = adj_matrix[(i, j)]
                adj_matrix[i, j] = -1  # 出流为负,改为入流为正
                # idx =j
                # 从 j 开始遍历，直到 n-1
                for idx in range(j, n):
                    if adj_matrix[i][idx] == -1:
                        adj_matrix[i][idx] = 1  # 这行代码实际上不会改变任何值，因为已经检查了等于1的条件
                        # break  # 找到值为1的元素后，跳出循环
                # if adj_matrix[i, j] < 0:  # 如果是从 i 流出到 j
                #     adj_matrix[i, j] = 1  # 出流为负,改为入流为正
                #     break

                # else:  # 如果是从 j 流入到 i
                #     A[i, idx] = 1  # 入流为正
print(adj_matrix)
# 使用numpy.linalg.solve求解
q = np.linalg.solve(adj_matrix,-node_demands)

print("解为：", q)
print("邻接阵：", adj_matrix)
adj_matrix[n-1,1]=0
# 指定的节点压力
# node_pressures = np.array([[10], [0],[0],[0],[0],[0]])  # 假设第一个节点的压力是已知的，其他节点的压力未知
node_pressures = np.array([0, 0,0,0,10,0])  # 假设第一个节点的压力是已知的，其他节点的压力未知
n = adj_matrix.shape[0]

print("A_1",adj_matrix)

A=adj_matrix

# 计算每行和每列的非零元素数量
row_nonzeros = (A != 0).sum(axis=1)
col_nonzeros = (A != 0).sum(axis=0)

# 寻找仅有一个非零元素的行和列
unique_row_indices = np.where(row_nonzeros == 1)
unique_col_indices = np.where(col_nonzeros == 1)

# 创建一个零矩阵 _like
A_new = np.zeros_like(A)
B_new = np.ones_like(A)
print(A_new)
# 将符合条件的位置设为1
for row_index in unique_row_indices:
    col_index = np.where(A[row_index, :] != 0) #>1) #
    if np.isin(col_index, unique_col_indices).any():  # 使用np.isin并结合.any()
        A_new[row_index, col_index] = 1
        B_new[row_index, col_index] = 0
    # else:
    #     A_new[row_index, col_index] = 0
A_new[unique_row_indices,0]=0
print("重写的A矩阵为:\n", A_new)
B_new[unique_row_indices,0]=1
print("重写的B矩阵为:\n", B_new)
# 使用 * 运算符计算Hadamard乘积
C_new = A * B_new
# C_new=np.dot(B_new,A)

print("重写的C矩阵为:\n", C_new)
Ac=C_new
Af=A_new
print("AF=",Af)
print("Ac",Ac)

#Yq -Ac TH c = A T f H f

resistances = np.array([
    [1, 0, 0, 0,0,0],
    [0, 2, 0, 0,0,0],
    [0, 0, 3, 0,0,0],
    [0, 0, 0, 4,0,0],
    [0, 0, 0, 0,1,0],
    [0, 0, 0, 0,0,0.20]
])

A=resistances*q-Ac.T

print(A)
print(Af.T)
b=Af.T*node_pressures

print(b)

H = spsolve(csr_matrix(A), b) #np.linalg.solve(A,b)

print("H 解为：", H)



解为： [ 1. -2. -2.]
方程组无解
[[-1 -1  0  0  0  0]
 [ 1  0  0  0  0  1]
 [ 0  0  0 -1  0  1]
 [ 0  0 -1  1  0  0]
 [ 0  1  1  0  0  0]
 [ 0  0  0  0  1  0]]
解为： [ 0.1 -0.  -0.  -0.1  0.  -0.1]
邻接阵： [[-1 -1  0  0  0  0]
 [ 1  0  0  0  0  1]
 [ 0  0  0 -1  0  1]
 [ 0  0 -1  1  0  0]
 [ 0  1  1  0  0  0]
 [ 0  0  0  0  1  0]]
A_1 [[-1 -1  0  0  0  0]
 [ 1  0  0  0  0  1]
 [ 0  0  0 -1  0  1]
 [ 0  0 -1  1  0  0]
 [ 0  1  1  0  0  0]
 [ 0  0  0  0  1  0]]
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
重写的A矩阵为:
 [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 1 0]]
重写的B矩阵为:
 [[1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 0 1]]
重写的C矩阵为:
 [[-1 -1  0  0  0  0]
 [ 1  0  0  0  0  1]
 [ 0  0  0 -1  0  1]
 [ 0  0 -1  1  0  0]
 [ 0  1  1  0  0  0]
 [ 0  0  0  0  0  0]]
AF= [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 1 0]]
Ac [[-1 -1  0  0  0  0]
 

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13120\1096868463.py:130: MatrixRankWarning: Matrix is exactly singular
  H = spsolve(csr_matrix(A), b) #np.linalg.solve(A,b)
